In [1]:
from sklearn.linear_model import LinearRegression
from statsmodels.stats.diagnostic import het_white
import statsmodels.api as sm
import pandas as pd
import numpy as np
import os

In [2]:
df = pd.read_excel('df_filtred_with_coordinates.xlsx')
df = df.loc[(df['preco']>49999)&(df['area util por m²']>9)]

In [3]:
# 1 - Alta
# 2 - Média
# 3 - baixa

classe_bairros = {
    'Pontal': 1,
    'Nossa Senhora da Vitória': 2,
    'São Francisco': 1,
    'São Sebastião': 2,
    'Jardim Savóia': 2,
    'Conquista': 2,
    'Centro': 1,
    'Olivença': 2,
    'Boa Vista': 3,
    'Jardim Atlântico': 1,
    'Cidade Nova': 2,
    'Ilhéus II': 2,
    'São Domingos': 2,
    'Teresópolis': 3,
    'Malhado': 2,
    'Nelson Costa': 2,
    'Esperança': 2,
    'Aritaguá': 3,
    'Hernani Sá': 2,
    'Iguape': 3
}

df['classe'] = df['bairro'].map(classe_bairros)

In [4]:
for col in df.columns:
    df[col] = df[col].replace(-1, 0)

df['log_preco'] = np.log(df['preco'])
df['log_area_util'] = np.log(df['area util por m²'])
df['log_distance_ceps'] = np.log(df['distance_between_ceps_in_meters'])

df.drop(columns=['custo condominio','iptu', 'preco', 'area util por m²', 'distance_between_ceps_in_meters',
                 'rua/avenida','cidade', 'estado', 'cep', 'cep_origin', 'latitude_origin', 'longitude_origin',
                 'latitude_destino', 'longitude_destino'], inplace=True)

In [5]:
# Tirando o centro pois não é algo viavel
df = df[df['cep_dest']!="Centro"]

In [6]:
'''lm = LinearRegression()

for bairro in df_zona_sul['bairro'].unique():
    
    if bairro not in ['Ilhéus II', 'Nelson Costa', 'Hernani Sá']:
        for infra in df_zona_sul['cep_dest'].unique():
            df_treated = df_zona_sul.loc[
                (df_zona_sul['bairro'] == bairro) & (df_zona_sul['cep_dest'] == infra)
            ].drop(columns=['bairro', 'cep_dest'])
            
            y = df_treated['log_preco']
            X = df_treated.drop(['log_preco'], axis=1)
            
            if len(df_treated) > 2:
                
                X = sm.add_constant(X)

                model = sm.OLS(y, X).fit()

                print(f'Regressão para Bairro: {bairro} e Infraestrutura: {infra}')
                print(model.summary())

                white_test = het_white(model.resid, model.model.exog)

                print(f'\nTeste de White para Heterocedasticidade:')
                print(f'Estatística: {white_test[0]}')
                print(f'p-value: {white_test[1]}\n')'''

"lm = LinearRegression()\n\nfor bairro in df_zona_sul['bairro'].unique():\n    \n    if bairro not in ['Ilhéus II', 'Nelson Costa', 'Hernani Sá']:\n        for infra in df_zona_sul['cep_dest'].unique():\n            df_treated = df_zona_sul.loc[\n                (df_zona_sul['bairro'] == bairro) & (df_zona_sul['cep_dest'] == infra)\n            ].drop(columns=['bairro', 'cep_dest'])\n            \n            y = df_treated['log_preco']\n            X = df_treated.drop(['log_preco'], axis=1)\n            \n            if len(df_treated) > 2:\n                \n                X = sm.add_constant(X)\n\n                model = sm.OLS(y, X).fit()\n\n                print(f'Regressão para Bairro: {bairro} e Infraestrutura: {infra}')\n                print(model.summary())\n\n                white_test = het_white(model.resid, model.model.exog)\n\n                print(f'\nTeste de White para Heterocedasticidade:')\n                print(f'Estatística: {white_test[0]}')\n                pri

In [10]:
# Aqui é para somente a zona sul
columns_not_used = ['Zona', 'bairro', 'cep_dest']
df_zona_sul = df[df['Zona']=='Sul'].drop(columns=['Zona'])

output_folder = 'resultado_modelos/zona sul'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

lm = LinearRegression()

for bairro in df_zona_sul['bairro'].unique():
    if bairro not in ['Ilhéus II', 'Nelson Costa', 'Hernani Sá']:
        for infra in df_zona_sul['cep_dest'].unique():
            df_treated = df_zona_sul.loc[
                (df_zona_sul['bairro'] == bairro) & (df_zona_sul['cep_dest'] == infra)
            ].drop(columns=['bairro', 'cep_dest'])
            
            y = df_treated['log_preco']
            X = df_treated.drop(['log_preco'], axis=1)
            
            if len(df_treated) > 2:
                X = sm.add_constant(X)

                model = sm.OLS(y, X).fit()

                

                summary_df = pd.read_html(model.summary().tables[1].as_html(), header=0, index_col=0)[0]
                
                white_test = het_white(model.resid, model.model.exog)
                white_test_results = pd.DataFrame({
                    'Estatística': [white_test[0]],
                    'p-value': [white_test[1]]
                }, index=['White Test'])
                
                r_squared = model.rsquared
                r_squared_adj = model.rsquared_adj
                r_squared_df = pd.DataFrame({
                    'R²': [r_squared],
                    'R² Ajustado': [r_squared_adj]
                }, index=['Estatísticas'])

                file_name = f"Regressao_{bairro}_{infra}.xlsx"
                output_path = os.path.join(output_folder, file_name)

                with pd.ExcelWriter(output_path) as writer:
                    summary_df.to_excel(writer, sheet_name='Regressao_Resultados')
                    white_test_results.to_excel(writer, sheet_name='Teste_White')
                    r_squared_df.to_excel(writer, sheet_name='R²_Estatisticas')

C:\Users\joaov\AppData\Local\Temp\ipykernel_23184\2354469325.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  summary_df = pd.read_html(model.summary().tables[1].as_html(), header=0, index_col=0)[0]
C:\Users\joaov\AppData\Local\Temp\ipykernel_23184\2354469325.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  summary_df = pd.read_html(model.summary().tables[1].as_html(), header=0, index_col=0)[0]
C:\Users\joaov\AppData\Local\Temp\ipykernel_23184\2354469325.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  summary_df = pd.read_html(model.summary().tables[1].as_html(), header=0, index_col=0)[0]
C:\Users\joaov\A

In [9]:
# Para as zonas
# Zona rural e extremo norte foram descartadas deveido a quantidade baixa de amostras
df_p_zonas = df.drop(columns=['bairro'])

output_folder = 'resultado_modelos/zonas'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

lm = LinearRegression()

for zona in df_p_zonas['Zona'].unique():
    if zona not in ['Rural']:
        for infra in df_p_zonas['cep_dest'].unique():
            df_treated = df_p_zonas.loc[
                (df_p_zonas['Zona'] == zona) & (df_p_zonas['cep_dest'] == infra)
            ].drop(columns=['Zona', 'cep_dest'])
            
            y = df_treated['log_preco']
            X = df_treated.drop(['log_preco'], axis=1)
            
            if len(df_treated) > 2:
                X = sm.add_constant(X)

                model = sm.OLS(y, X).fit()

                summary_df = pd.read_html(model.summary().tables[1].as_html(), header=0, index_col=0)[0]
                
                white_test = het_white(model.resid, model.model.exog)
                white_test_results = pd.DataFrame({
                    'Estatística': [white_test[0]],
                    'p-value': [white_test[1]]
                }, index=['White Test'])
                
                r_squared = model.rsquared
                r_squared_adj = model.rsquared_adj
                r_squared_df = pd.DataFrame({
                    'R²': [r_squared],
                    'R² Ajustado': [r_squared_adj]
                }, index=['Estatísticas'])

                file_name = f"Regressao_{zona}_{infra}.xlsx"
                output_path = os.path.join(output_folder, file_name)

                with pd.ExcelWriter(output_path) as writer:
                    summary_df.to_excel(writer, sheet_name='Regressao_Resultados')
                    white_test_results.to_excel(writer, sheet_name='Teste_White')
                    r_squared_df.to_excel(writer, sheet_name='R²_Estatisticas')

C:\Users\joaov\AppData\Local\Temp\ipykernel_23184\3288313224.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  summary_df = pd.read_html(model.summary().tables[1].as_html(), header=0, index_col=0)[0]
C:\Users\joaov\AppData\Local\Temp\ipykernel_23184\3288313224.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  summary_df = pd.read_html(model.summary().tables[1].as_html(), header=0, index_col=0)[0]
C:\Users\joaov\AppData\Local\Temp\ipykernel_23184\3288313224.py:26: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  summary_df = pd.read_html(model.summary().tables[1].as_html(), header=0, index_col=0)[0]
C:\Users\joaov\A